# 1. Get city outlines and hexbins them

In this notebook, we will be using the Global Human Setllement Layer's [Urban Center Database](https://human-settlement.emergency.copernicus.eu/ghs_stat_ucdb2015mt_r2019a.php) to select which cities we will analyze. We will also divide the selected cities into hexagons using Uber's [H3 package](https://www.uber.com/en-BR/blog/h3/).

In [1]:
# Importing packages
import ee
import geemap
import geopandas as gpd
import h3
from os import makedirs
from os.path import exists
import pandas as pd
from unidecode import unidecode
from shapely.geometry import Polygon

In [2]:
# Setting options
pd.set_option('display.max_rows', 500)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

## 1.1 Selecting urban centers

We are using the outlines of the urban centers as defined on this [research paper](https://www.nature.com/articles/s41597-023-02691-1), by scientists at the EU'S Joint Research Center. It uses satellite observations to estimate population density at the entire Earth. Then, from this observations, it derives the extent of urban centers. The outline of cities _will not_ coindice with political-administrative divisions. Sometimes, one urban center may extend beyond national borders or connect cities that are often thought of as separate entities. However, as the article states, this approach is still best for international comparisons than relying on hard-to-compare, arbitrary administrative boundaries.

In [3]:
# Urban centers as defined by the EU Joint Research Center
cities = gpd.read_file("../data/GHS_STAT_UCDB2015MT_GLOBE_R2019A/GHS_STAT_UCDB2015MT_GLOBE_R2019A_V1_2.gpkg")

In [4]:
# Gets rid of urban centers marked as 'false positives' and 'uncertain'
# This means that, while the satellite detected something there, experts disagreed about
# whether it should be classified as such.
cities = cities[cities.QA2_1V==1]

In [5]:
# Applies unidecode on the city names to avoid errors 
# when saving files on Google Earth Engine later
cities['UC_NM_LST'] = cities['UC_NM_LST'].apply(unidecode)
cities['UC_NM_MN'] = cities['UC_NM_MN'].apply(unidecode)

In [6]:
# Keeps only the necessary column names
cities = cities[['ID_HDC_G0', 'CTR_MN_NM', 'UC_NM_MN', 'CTR_MN_ISO', 'UC_NM_LST', 'P15', 'geometry']]

In [7]:
# Saves the file as a shapefile.
# This shapefile will be uploaded manually to Google Earth Engine
# under the name 'projects/dw-city-tree-coverage/assets/city_outlines'
dir_ = "../output/city_outlines"
if not exists(dir_):
    makedirs(dir_)
    
cities.to_file(f"{dir_}/city_outlines.shp")

## 1.2  Selecting key urban centers and making H3 cells

For some key urban centers (capitals and large cities of countries we are specially interested in), we will split the geometries into smaller areas. This way, we can compute not only the total tree coverage of each city, but the tree dsitribution within different areas of cities. It also will allow us to compute how many people live in areas with high tree coverage and low tree coverage within the same city.

The areas will be divided into hexagons using the H3 package because of its ease of use, compared to alternatives such as S2, which would produce squaes. In a personal note, hexagons also look better than squares when plotted over maps.

One important thing to notice is that the hexagons will _not_ have all the same area. They will vary according to latitude, ranging from around 600sqm to 1000sqm – with most of them falling on the 700–800sqm range. Nevertheless, _within a single city_, they will always be of comaparable size.

In [8]:
# A manual selection of the cities we are interested in
cities_of_interest = [6118, 3050, 3274, 5915, 1105, 3027, 5528, 2135, 2101, 9872,
3381, 3196, 849, 1303, 2384, 931, 10687, 12080, 12274, 3209,
621, 392, 473, 4094, 2927, 2851, 2749, 816, 2095,
467, 3902, 1701, 3255, 5134, 3244, 2046, 1910, 1502, 3420,
288, 359, 3032, 741, 3193, 11862, 6955, 1656, 5737, 5314,
1510, 2897, 4408, 12964, 5936, 4808, 12389, 4337, 2971,
8835, 3339, 2253, 3269, 1575, 143, 3332, 3034, 10248, 4220,
11139, 2125, 367, 2167, 2493, 6169, 458, 528, 12829, 3216,
1551, 1074, 3444, 3675, 4172, 5698, 4335, 1452, 11498, 321,
3270, 4322, 3069, 2965, 2973, 4407, 3186, 10715, 3809, 12696,
4679, 4427, 3517, 1138, 10, 79, 183, 323, 400,
530, 634, 651, 770, 855, 945, 1022, 1045, 6008, 857,
11136, 3323, 5635, 3673, 3868, 3798, 367, 2971]

In [9]:
# A quick look at the cities listes above
for index, row in cities[cities.ID_HDC_G0.isin(cities_of_interest)].iterrows():
    print(row.UC_NM_MN, '-', row.CTR_MN_NM)

San Jose - United States
Phoenix - United States
Ciudad Juarez - Mexico
Anchorage - United States
Guatemala City - Guatemala
San Salvador - El Salvador
Dallas - United States
Tegucigalpa - Honduras
Managua - Nicaragua
San Jose - Costa Rica
New Orleans - United States
Panama City - Panama
Quito - Ecuador
Havana - Cuba
Lima - Peru
Nashville - United States
Bogota - Colombia
Chicago - United States
Indianapolis - United States
Port-au-Prince - Haiti
Detroit - United States
Santo Domingo - Dominican Republic
El Alto [La Paz] - Bolivia
Washington D.C. - United States
Caracas - Venezuela
Santiago - Chile
New York - United States
Boston - United States
Portland - United States
Asuncion - Paraguay
Buenos Aires - Argentina
Montevideo - Uruguay
Sao Paulo - Brazil
Dakar - Senegal
Conakry - Guinea
Reykjavik - Iceland
Lisbon - Portugal
Casablanca - Morocco
Dublin - Ireland
Madrid - Spain
Accra - Ghana
Paris - France
Algiers - Algeria
Porto-Novo - Benin
Lagos - Nigeria
Brussels - Belgium
Amsterdam -

In [10]:
# Creates a new gdf with those cities only.
# Not that we are resetting the index, so they will align with a for loop later.
hexbinned_cities = cities[cities.ID_HDC_G0.isin(cities_of_interest)].reset_index()

In [11]:
# Converts all of them to H3 cells at resolution 8.
# They will have roughly 0.73sqkm, according to the documentation.
# https://h3geo.org/docs/core-library/restable/
# Notice, again, the reset index
city_hexagons = hexbinned_cities.geometry.apply(lambda x: h3.geo_to_cells(x, res=8)).reset_index()

In [12]:
# We now have hexadecimal ids for all the hexagons within a city.
# For example, city at index 9 was decomposed in hexagons 882834443bfffff, 8828347307fffff... and so on.
# We need to convert that into something that looks more like a proper Polygon.
city_hexagons.head()

,index,geometry
0,0,"[882834443bfffff, 8828347307fffff, 882830983bf..."
1,1,"[8848ebaa85fffff, 8829b6d431fffff, 8829b6da07f..."
2,2,"[8848c6c653fffff, 8848c68981fffff, 8848c68b37f..."
3,3,"[880c733a1dfffff, 880c733a89fffff, 880c733a11f..."
4,4,"[886d291a81fffff, 886d290319fffff, 886d290143f..."


In [13]:
# For each city...
all_entries = []
for index, row in city_hexagons.iterrows():
    
    # Acces its hex
    hexagons = row.geometry
    
    # For each hexagon, create a shape and get relevant information
    for i, hexagon in enumerate(hexagons):
        # This is in lat/lon. Shapely need it in lon/lat
        bounds = h3.cell_to_boundary(hexagon)
        new_bounds = []
        for bound in bounds:
            lon = bound[1]
            lat = bound[0]
            new_bound = (lon, lat)
            new_bounds.append(new_bound)
        
        # Creates a shapely Polygon out of the new bounds
        poly = Polygon(new_bounds)
        
        # Creates an entry with extra information to save on the list
        entry = {
            'geometry': poly, # A shapely polygon constructed out of the H3 cell
            'area': h3.cell_area(hexagon), # The area of the hexagon in sqm
            'city_id': hexbinned_cities.loc[index, 'ID_HDC_G0'], # The city id
            'hexagon_n': i # An unique number for each hexagon in a city
        }
        
        all_entries.append(entry)

In [14]:
# Converts the list of dictionaries we just made into a geodataframe,
# replacing the one with the simple city oulines.
# This will have no CRS, as we were working with naive geometries.
hexbinned_cities = gpd.GeoDataFrame(all_entries)

In [15]:
# Sets the results to the same crs as the city outlines
hexbinned_cities.crs = cities.crs

In [18]:
# Saves it locally.
# under the name 'projects/dw-city-tree-coverage/assets/city_hexagons'
dir_ = "../output/city_hexagons"
if not exists(dir_):
    makedirs(dir_)
hexbinned_cities.to_file(f"{dir_}/city_hexagons.shp")